In [ ]:
# First, we ensure we have no other jobs, then declare our outputs and submit. 
# When running, the scheduler will ignore the 'scheduleNotebook' function
hise::clearNotebookJob()
list_data <- list("Seurat_output.RData")
hise::scheduleNotebook(outputFiles=list_data, platform="Seurat", project="aifiDevTest")

In [ ]:
# Let's ensure SeuratDisk and tidyverse are installed
system("apt-get update && apt-get install libhdf5-dev -y && PATH=\"/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin\" Rscript -e \"remotes::install_github('mojaveazure/seurat-disk')\"")
install.packages('tidyverse')

In [ ]:
library(Seurat)
library(SeuratDisk)
library(tidyverse)
library(scattermore)
library(Matrix)

In [ ]:
# Then, we'll read in the h5's we intend to use
hise::readFiles(list("676faa6b-df1a-45a0-ad91-0320bdb23f00", "66c15b4a-c331-4222-b723-51d67f241de6", "522c5ac1-1ddd-42fb-a938-09fdaf2cf70f"))

In [ ]:
# As well as our reference set
reference <- LoadH5Seurat("/tmp/refs/pbmc_multimodal.h5seurat")

In [ ]:
# We'll load our h5's from disk into a list
h5files<-list.files(pattern="h5",path = "/home/jupyter/cache",recursive = T,full.names = T)
h5list<-lapply(h5files,function(file){
    H5weaver::read_h5_seurat(file,feature_names = "name")
})

In [ ]:
# And then merge them, and clean up
so<-merge(h5list[[1]],h5list[2:length(h5list)])
rm(h5list)
rm(h5files)
gc()

In [ ]:
# We'll run SCTransform
options(warn=-1)
so <- SCTransform(so, conserve.memory = T)
options(warn=0)

In [ ]:
anchors <- FindTransferAnchors(
  reference = reference,
  query = so,
  normalization.method = "SCT",
  reference.reduction = "spca",
  dims = 1:50
)
so <- MapQuery(
  anchorset = anchors,
  query = so,
  reference = reference,
  refdata = list(
    celltype.l1 = "celltype.l1",
    celltype.l2 = "celltype.l2",
    celltype.l3 = "celltype.l3",      
    predicted_ADT = "ADT"
  ),
  reference.reduction = "spca", 
  reduction.model = "wnn.umap"
)

In [ ]:
so<-RunPCA(so)
so<-RunUMAP(so,dims=1:50)

In [ ]:
# Save as RData? 
save(so, anchors, file = "Seurat_output.RData")